In [ ]:
from clean_data import run_cleaning_pipeline
from convert_parquet_to_json import convert_parquet_to_json
from mongo_import import import_json_to_mongodb
from logger import logger
#import os


In [ ]:
INPUT_PATH = "../data/raw/fhvhv_tripdata_2021-10.parquet"
JSON_PATH = "../data/processed/trips.json"

columns_to_remove =[
    'originating_base_num', 
    'on_scene_datetime',
    'access_a_ride_flag'
]

columns_clean = [
    'hvfhs_license_num',
    'dispatching_base_num'
]

flag_cols = [
    'shared_request_flag',
    'shared_match_flag', 
    'wav_request_flag',	
    'wav_match_flag'
]

DB_NAME = "trips_db"
COLLECTION_NAME = "fhvhv_trips_2021-10"



In [ ]:
# print(os.getcwd())
# print(os.path.exists(INPUT_PATH))

In [ ]:
# Step 1: Clean the data and get a DataFrame
df = run_cleaning_pipeline(INPUT_PATH, columns_to_remove, columns_clean, flag_cols)

In [ ]:
print(df)


In [ ]:
df1 = df.iloc[0:6_000_000]

In [ ]:
l = len(df)
print(df.iloc[l-1:l])

In [ ]:
i = 0
batch_size = 1_000_000
count = 0

In [ ]:
# Step 2: Convert the cleaned DataFrame to JSON Lines format

while i+batch_size < l:
    batch = df.iloc[i:i+batch_size]
    JSON_PATH = f"../data/processed/trips{count}.json"
    convert_parquet_to_json(batch, JSON_PATH)
    i += batch_size 
    count +=1
if i < l:
    batch = df.iloc[i:l]
    JSON_PATH = f"../data/processed/trips{count}.json"
    convert_parquet_to_json(batch, JSON_PATH)

In [ ]:
#test 1 json
JSON_PATH = f"../data/processed/trips0.json"
import_json_to_mongodb(JSON_PATH, DB_NAME, COLLECTION_NAME)

In [ ]:
# Step 3: Import the JSON Lines data into MongoDB
count = 16
for c in range(count):
    JSON_PATH = f"../data/processed/trips{c}.json"
    import_json_to_mongodb(JSON_PATH, DB_NAME, COLLECTION_NAME)
    print(f'json{c}')